In [12]:
#v2.0 batch mode

#chdir
import os,re,time
os.chdir("/home/wangjl/data/apa/190705PAS/")
os.getcwd()

'/data/jinwf/wangjl/apa/190705PAS'

In [9]:
# no use this box
#read cell-IDs from file 

cidFile=['/home/wangjl/data/apa/190530Mix/cluster2_syncHeLa.27L.cellID', #12+15个细胞是同步化的hela，来自于left
'/home/wangjl/data/apa/190530Mix/cluster2_HeLa_nonsyncHeLa.29R.cellID', #29个非同步化的Hela，来自于right

'/home/wangjl/data/apa/190530Mix/cluster0_BC.87.cellID', #c0和c1为hela，都来自于right
'/home/wangjl/data/apa/190530Mix/cluster1_BC.82.cellID']

def getIDfromFIle(cidFile):
    fr=open(cidFile,"r")
    cids=[]
    for lineR in fr.readlines():
        line=lineR.strip()
        cids.append(line)

    fr.close()
    return cids;

cids=getIDfromFIle(cidFile[3])
print(len(cids),cids)

82 ['c12_ROW02', 'c12_ROW04', 'c12_ROW07', 'c12_ROW11', 'c12_ROW20', 'c12_ROW24', 'c12_ROW25', 'c12_ROW31', 'c12_ROW33', 'c12_ROW34', 'c12_ROW35', 'c12_ROW36', 'c12_ROW38', 'c13_ROW06', 'c13_ROW09', 'c13_ROW15', 'c13_ROW26', 'c13_ROW29', 'c13_ROW32', 'c13_ROW34', 'c13_ROW36', 'c14_ROW05', 'c14_ROW07', 'c14_ROW08', 'c14_ROW09', 'c14_ROW15', 'c14_ROW16', 'c14_ROW18', 'c14_ROW19', 'c14_ROW22', 'c14_ROW23', 'c14_ROW24', 'c14_ROW25', 'c14_ROW26', 'c14_ROW28', 'c14_ROW29', 'c14_ROW30', 'c14_ROW37', 'c14_ROW39', 'c15_ROW06', 'c15_ROW12', 'c15_ROW13', 'c15_ROW16', 'c15_ROW18', 'c15_ROW19', 'c15_ROW26', 'c15_ROW28', 'c15_ROW31', 'c15_ROW33', 'c15_ROW34', 'c15_ROW35', 'c15_ROW38', 'c16_ROW02', 'c16_ROW03', 'c16_ROW05', 'c16_ROW08', 'c16_ROW09', 'c16_ROW11', 'c16_ROW12', 'c16_ROW14', 'c16_ROW23', 'c16_ROW24', 'c16_ROW25', 'c16_ROW28', 'c16_ROW31', 'c16_ROW34', 'c16_ROW37', 'c16_ROW38', 'c19_ROW04', 'c19_ROW07', 'c19_ROW09', 'c19_ROW10', 'c19_ROW13', 'c19_ROW15', 'c19_ROW17', 'c19_ROW18', 'c19_ROW

In [16]:
#{cut polyA} input cell id, keep reads with 10A
def getPolyAReadCutA(cid,fromDir="bc"):
    basePath="/home/wangjl/data/apa/fq_files/"+fromDir+"/"
    cellPath=basePath+cid+"_R2.fastq"
    #print(cellPath) # /home/wangjl/data/apa/fq_files/bc/c12_ROW03_R2.fastq

    # read file
    fr=open(cellPath,"r")
    # write to files: 3, 10A /end 6A /other
    fw10A=open("/home/wangjl/data/apa/190705PAS/polyA_fq/CutA_"+cid+".10A.fq", 'w')
    #fwTail6A=open("/home/wangjl/data/apa/190610APA/polyA_fq/CutA_"+cid+".Tail6A.fq", 'w')
    fwOther=open("/home/wangjl/data/apa/190705PAS/polyA_fq/CutA_"+cid+".Other.fq", 'w')

    i=0
    fastq=[] #fq file 4 row
    tag="" #tag for each 4 row
    start=time.time()
    all_lines=fr.readlines()
    for lineR in all_lines:
        i+=1
        #if i>100:
        #    print("=============debug model break here;");break; #for test
        #
        #if i% round(len(all_lines)/10)==0:
        #    print(i, time.time()-start,"seconds elapsed.")

        #
        line=lineR.strip()
        fastq.append(lineR)

        #give tag by 2nd row
        if i%4==2:
            #print(i,":",line)
            haveA10=re.search("A{10,}",line)
            haveA6=re.search("A{6,}$",line)
            #
            if haveA10:
                tag="10A"
            elif haveA6:
                tag="Tail6A"
            else:
                tag="Other"

        # at row 4, output by tag
        if i%4==0:
            #print(i/4,":",tag,":",fastq[1].strip())
            # save to mem by tag at row 1,3
            if tag=="10A" and (haveA10.span()[0]>10):
                #print(haveA10.span()[0],"\n")
                fastq[1]=fastq[1].strip()[0: haveA10.span()[0] ]+"\n"
                fastq[3]=fastq[3].strip()[0: haveA10.span()[0] ]+"\n"
                for item in fastq:
                    fw10A.write(item)
            elif tag=="Tail6A" and (haveA6.span()[0]>10):
                #print(haveA6.span()[0],"\n")
                fastq[1]=fastq[1].strip()[0: haveA6.span()[0] ]+"\n"
                fastq[3]=fastq[3].strip()[0: haveA6.span()[0] ]+"\n"
                for item in fastq:
                    #fwTail6A.write(item)
                    fw10A.write(item)
            else:
                # too short, or no 10A or 6A
                for item in fastq:
                    fwOther.write(item)

             #clen
            fastq=[]
            
    #close fiels.
    fr.close()
    fw10A.close()
    #fwTail6A.close()
    fwOther.close()
    return 0;

# cut off polyA
cid="c01_ROW24"
#for cid in cids:
print('===Cut-start===', cid)
getPolyAReadCutA(cid,"mef")
print("===Cut-end===",cid)

===Cut-start=== c01_ROW24
===Cut-end=== c01_ROW24


In [17]:
# parallel
################################
#part 3 parallel
################################
import time,multiprocessing,os,re #random
from multiprocessing import Queue
from multiprocessing import Process

#=====================
#settings
#id_list="Right.cellID"
id_list="/home/wangjl/data/apa/190530Mix/cluster2_syncHeLa.27L.cellID"
# input file name of id list, one id a row. Used in doLinuxCMD()
# default below
core=20 # CPU core number
hint_n=5 # default hint number
log_file_name='cutA-logs.txt' #uniq-count-FROM-star.final.207.log
# log file names. contents: #id status(0 normal, other error) 
#=====================
## add time stamp to log file name, in case overwrite.
timsString=time.strftime("%Y%m%d-%H%M%S", time.localtime()) 
log_file_name=log_file_name + timsString


# read and process, parallelly
def worker(cb):
    #time consuming process
    #rs=cb+"\t"+str( getPolyAReadCutA(cb) ) #defined in part2
    rs=cb+"\t"+str( getPolyAReadCutA(cb,'mef') )  # input cell id and dir/
    q.put(rs)   #output to queue

# save thread, only one
def writer(log_file_name,ID_total,hint_n=20):
    breaks=int(ID_total/hint_n) 
    i=0
    with open(log_file_name, 'w') as f:
        while True:
            i+=1
            if i%breaks==0:
                print(i," items processed in ", time.time()-start," seconds",sep="")
            rs=q.get() #waite while q is empty
            f.write(rs+"\n")
            f.flush()


#start time
start=time.time()
print('='*10, ">Begin of main process[", os.getpid(), "][child pid by parent ppid]", sep="")

# main thread
if __name__ == '__main__': 
    q=Queue(core+10)
    pool = multiprocessing.Pool(processes = core)

    # commit task to pool, and worker() parallel
    fr=open(id_list,'r')# read id_list
    i=0
    for lineR in fr.readlines():
        i+=1
        #if i>100:
        #    pass;
            #break;

        line=lineR.strip()
        arr=re.split(' ',line)
        pool.apply_async( worker,args=(arr[0],) )
    fr.close() #

    # write thread from queue
    pOut = Process(target=writer, args=(log_file_name,i,hint_n,))
    pOut.start()

    # waite for end of worker()
    pool.close()
    pool.join()

    while not q.empty():
        time.sleep(1)#check queue every second
    pOut.terminate();

print(time.time()-start,'s <', '='*10, "End of main process[", os.getpid(),']', sep="")
print("#"*40,"\nLog file: "+ os.getcwd()+"/"+log_file_name)

==========>Begin of main process[439947][child pid by parent ppid]
5 items processed in 87.45579409599304 seconds
10 items processed in 100.83897280693054 seconds
15 items processed in 111.21550917625427 seconds
20 items processed in 122.79621720314026 seconds
25 items processed in 153.1750123500824 seconds
217.95235681533813s <==========End of main process[439947]
######################################## 
Log file: /data/jinwf/wangjl/apa/190705PAS/cutA-logs.txt20190712-035309
